In [2]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.datasets import fetch_lfw_people
import matplotlib.pyplot as plt


In [14]:
def pca(X, k):
    # Step 1: Standardize the data
    X_std = (X - np.mean(X, axis=0)) / np.std(X, axis=0)

    # Step 2: Compute the covariance matrix
    cov_matrix = np.cov(X_std.T)

    # Step 3: Compute the eigenvectors and eigenvalues
    eigenvalues, eigenvectors = np.linalg.eig(cov_matrix)
    eigenvectors = np.real(eigenvectors)
    eigenvalues = np.real(eigenvalues)

    # Step 4: Sort the eigenvectors
    eigenvectors = eigenvectors[:, np.argsort(eigenvalues)[::-1]]

    # Step 5: Select the top k eigenvectors
    top_k_eigenvectors = eigenvectors[:, :k]

    # Step 6: Project the data onto the new feature space
    X_pca = np.dot(X_std, top_k_eigenvectors)

    # Calculate explained variance ratio
    explained_variance_ratio = eigenvalues[:k] / np.sum(eigenvalues)

    return X_pca, explained_variance_ratio

In [3]:
lfw_people = fetch_lfw_people(min_faces_per_person=70, resize=0.4)
n_samples, h, w = lfw_people.images.shape
X = lfw_people.data
y = lfw_people.target
target_names = lfw_people.target_names
n_classes = target_names.shape[0]

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)


In [16]:
# Compute PCA (eigenfaces) on the face dataset 
X_train_pca,variance_ration= pca(X_train, 150)
X_test_pca= pca(X_test, 150)



In [17]:
# Train a SVM classification model
clf = SVC(kernel='rbf', class_weight='balanced')
clf = clf.fit(X_train_pca, y_train)

# Quantitative evaluation of the model quality on the test set
y_pred = clf.predict(X_test_pca)
print(classification_report(y_test, y_pred, target_names=target_names))

# Plot the explained variance ratio
plt.figure()
plt.plot(np.cumsum(variance_ration))
plt.xlabel('Number of Components')
plt.ylabel('Variance (%)') # for each component
plt.title('Explained Variance')
plt.show()

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (2,) + inhomogeneous part.